In [12]:
import os
import random
import shutil
from pathlib import Path
import xml.etree.ElementTree as ET

from ultralytics import YOLO

### Converting XML files into TXT

In [13]:
DATA_PATH = Path("../data/labels")


def convert(size, box):
        dw = 1. / size[0]
        dh = 1. / size[1]
        x = (box[0] + box[1]) / 2.0 - 1
        y = (box[2] + box[3]) / 2.0 - 1
        w = box[1] - box[0]
        h = box[3] - box[2]
        x = x * dw
        w = w * dw
        y = y * dh
        h = h * dh
        return (x, y, w, h)


if not DATA_PATH.exists():

    input_dir = "../data/annotations"           # path to the directory containing the xml files
    output_dir = "../data/labels"               # path to the directory where the txt files will be saved
    images_dir = "../data/images"               # path to the directory containing the images

    # create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # iter through all the xml files in the input directory
    for filename in os.listdir(input_dir):
        if not filename.endswith(".xml"):
            continue
        
        # parse the xml file
        tree = ET.parse(os.path.join(input_dir, filename))
        root = tree.getroot()

        # get the image size
        image_size = root.find("size")
        width = int(image_size.find("width").text)
        height = int(image_size.find("height").text)

        # create the txt file with the same name as the image file and save it in appropriate format
        with open(os.path.join(output_dir, filename.replace(".xml", ".txt")), "w") as out_file:
            for obj in root.iter("object"):
                cls = obj.find("name").text
                if cls != "licence":  
                    continue
                cls_id = 0  # hardcoding class id for licence plate

                xmlbox = obj.find("bndbox")
                b = (float(xmlbox.find("xmin").text), float(xmlbox.find("xmax").text),
                    float(xmlbox.find("ymin").text), float(xmlbox.find("ymax").text))
                bb = convert((width, height), b)
                out_file.write(f"{cls_id} " + " ".join([str(a) for a in bb]) + '\n')


### Splitting dataset into training set and validation set

In [14]:
image_dir = "../data/images/"
label_dir = "../data/labels/"

train_ratio = 0.8

images = [f for f in os.listdir(image_dir) if f.endswith(".png")]
random.shuffle(images)

train_split = int(len(images) * train_ratio)
train_images = images[:train_split]
val_images = images[train_split:]

# Przeniesienie plików
for img in train_images:
    shutil.move(os.path.join(image_dir, img), os.path.join(image_dir, "train", img))
    shutil.move(os.path.join(label_dir, img.replace(".png", ".txt")), os.path.join(label_dir, "train", img.replace(".png", ".txt")))

for img in val_images:
    shutil.move(os.path.join(image_dir, img), os.path.join(image_dir, "val", img))
    shutil.move(os.path.join(label_dir, img.replace(".png", ".txt")), os.path.join(label_dir, "val", img.replace(".png", ".txt")))


### Fine-tuning of YOLO

In [15]:
model_path = Path("plates_YOLO.pt")

if not model_path.exists():
    model = YOLO("yolov5s.pt")
    results = model.train(data="../data/car_plates.yaml", 
                        epochs=100, 
                        project="yolov5s", 
                        name="yolov5s", 
                        exist_ok=True, 
                        device=0)
    
    model.save("plates_YOLO.pt")
    
else:
    model = YOLO(model_path)